In [ ]:
from holoviews.streams import PointDraw, PolyEdit, BoxEdit, PolyDraw
from ipywidgets import widgets, HBox, Label
from rasterio.io import MemoryFile
from itertools import product
from io import BytesIO
from PIL import Image
import cartopy.crs as ccrs
import holoviews as hv
import geoviews as gv
import numpy as np
import requests
import rasterio
import math
import sys
import os

hv.extension('bokeh')

In [ ]:
url = 'http://c.tile.openstreetmap.org/{Z}/{X}/{Y}.png'
tiles = gv.WMTS(url, extents=(-180, -90, 180, 90), crs=ccrs.PlateCarree()).options(width=450, height=450)
points = gv.Points([], crs=ccrs.PlateCarree()).options(tools=['hover'], size=5, color='red')
point_stream = PointDraw(source=points)
tiles * points

In [ ]:
my_switch = False
t = point_stream.element.dframe()
print(t)
if len(t) > 2 and len(t) == 0:
    my_switch = True
    print("To many points...!")
else:
    incoming_longs = t['x'].tolist()
    incoming_lats = t['y'].tolist()
    print("Long: ", incoming_longs)
    print("Lat: ", incoming_lats)
    if incoming_longs[0] > incoming_longs[1] and incoming_lats[0] < incoming_lats[1]:
        my_switch = True
    else:
        value = 0.5
        polygon = [[[incoming_longs[0], incoming_lats[0]], [incoming_longs[1], incoming_lats[0]], [incoming_longs[1], incoming_lats[1]], [incoming_longs[0], incoming_lats[1]]]]
        tiles2 = gv.WMTS(url, extents=(incoming_longs[0] + -(value), incoming_lats[1] + -(value), incoming_longs[1] + value, incoming_lats[0] + value), crs=ccrs.PlateCarree()).options(width=450, height=450)
        shape = gv.Polygons(polygon, crs=ccrs.PlateCarree()).options(fill_alpha=0.4, color='blue', line_color='blue', line_width=2)
    
tiles2 * shape

In [ ]:
zoom_widget_value = widgets.IntSlider(min=0,max=19,step=1,description='Zoom Level:',orientation='horizontal')
display(zoom_widget_value)

In [ ]:
%time
zoom_level = zoom_widget_value.value
url = 'http://c.tile.openstreetmap.org/{0}/{1}/{2}.png'
# url = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{0}/{1}/{2}'

# Get the basemap tiles for the given coordinates.
convereted_lat_tile_list = [int(1/(2*math.pi)*2**zoom_level*(math.pi-math.log(math.tan(math.pi
                                                /4+math.radians(y)/2)))) for y in incoming_lats]
convereted_long_tile_list = [int(2**(zoom_level-1)*(x/180+1)) for x in incoming_longs]

# Get the top left and bottom right hand corner coordinates of where the actual basemap tile starts.
# The top left hand corner of the bbox will not have the coordinates as the basemap tile because of 
# - them never going to line up perfectly.
# We have to increment the long and lat of the bottom right hand corner because we need to get the next
# - tile and caculate the coordinates of its top left hand corner.
# Why I am doing this is because we have to geo-reference the image and have the exact coordinates of the
# - actual image that we will stitch together from these tiles.
first_lat = math.degrees(2*(math.atan(math.exp(math.pi-(convereted_long_tile_list[0]*2*math.pi)
                                               /2**zoom_level))-math.pi/4))
second_lat = math.degrees(2*(math.atan(math.exp(math.pi-((convereted_long_tile_list[-1] + 1)*2*math.pi)
                                                /2**zoom_level))-math.pi/4))

first_long = math.degrees(convereted_lat_tile_list[0]*2*math.pi/(2**zoom_level) - math.pi)
second_long = math.degrees((convereted_lat_tile_list[-1] + 1)*2*math.pi/(2**zoom_level) - math.pi)

# Have to create a range item so that we can do a product on it later in the algorithm. So lets increment,
# - the last element in either list to include up to the next number. If we didn't then it would stop short
# - one number.
# Next I have to figure out the number of tiles for x and y, so that I can see how big the image is going 
# - to be.
lat_range_list = range(convereted_lat_tile_list[0], (convereted_lat_tile_list[1] + 1))
number_of_y_tiles = (lat_range_list[-1] + 1) - lat_range_list[0]

long_range_list = range(convereted_long_tile_list[0], (convereted_long_tile_list[1] + 1))
number_of_x_tiles = (long_range_list[-1] + 1) - long_range_list[0]

# This loop will create the image from the previous calculated size, by multiplying the number of tiles for
# - x and y by 256 (pixels). This is the standard size for every tile (256,256) for open street map. Next, 
# - I will do the product on the range of both the lat and long. This will give me all the tiles from the 
# - top left corner to the the bottom right. Next, I format the url with the given zoom_level, and calculated
# - tile number to download. If I get a good response then I check to see if we are on the first iteration, 
# - to set my min x and y tile numbers. I do this because I need the baseline to subtract all future tile 
# - numbers from in order to acurately know where each tile will go in the grid. This has to be done this way
# - because not all tiles will be retrieved in order, and this saves lines of code by not having to make sure
# - that the pictures are all in order. Next, I increment my iterator so I don't come back into the if 
# - statment, I open the image in memory, then subtract the baseline x and y from the current tile number,
# - and multiply by the number of pixels. Ex: (4 - 2) * 256 = 512 for x. This will put the picture right at 
# - the calculated number of pixels either over or down. Next I stitch the opened image to the one I created
# - at the beinging of the loop. This will hold all the images once they are all stitched in. Lastly, I close
# - the image that is opened in memory.
loop_iterator = 0
stitched_image = Image.new('RGB', (number_of_x_tiles * 256, number_of_y_tiles * 256 ))
for x, y in product(long_range_list, lat_range_list):
    response = requests.get(url.format(zoom_level, x, y), verify=False)
    if response.status_code == 200:
        if loop_iterator == 0:
            min_tile_index_width = x
            min_tile_index_height = y
            loop_iterator = 1

        incoming_image = Image.open(BytesIO(response.content))
        x = (x - min_tile_index_width) * 256
        y = (y - min_tile_index_height) * 256
        stitched_image.paste(im=incoming_image, box=(x, y))
        incoming_image.close()

# Here I will output the image to the desktop and not the browser. This is because the image can be several
# - 100 megabytes big, depending on the given zoom level. Next, I save the image to the current working directory.
# - Next, I load the image from memory into a numpy array, and close the image. I do this here becasue if the 
# - image is big then we can free up all of our memory for the numpy array. Next, I take the transform for our
# - geo-referenced image. The transform operation will take the calcuated coordinates of out top left and bottom
# - right tiles, and the width and height of the image. Next, I have to move the axis around so that our number
# - of bands is in front of the tuple, rather than at the back. Since our image is RGB, the number of bands is 3,
# - for three colors. Next, I take the image that is in the numpy array and open it with the rasterio call. Lastly,
# - I write the image to disk, and print out where the file was saved to and the size of the image.
stitched_image.show()
file_path = os.path.join(os.getcwd(), "Thy_Picture.tiff")
image_array = np.array(stitched_image)
stitched_image.close()
transform = rasterio.transform.from_bounds(first_long, second_lat, second_long, first_lat, number_of_x_tiles * 256, number_of_y_tiles * 256)
image_array = np.moveaxis(image_array, -1, 0)
with rasterio.open(file_path, 'w', driver='GTiff', height=image_array.shape[1], 
                width=image_array.shape[2], count=image_array.shape[0], dtype=image_array.dtype,
                crs='EPSG:3957', transform=transform) as dst:
    dst.write(image_array, indexes=[1, 2, 3])

print(file_path)

In [ ]:
my_string = "89.99, 41.303, 80.039, 45.058"
parts = my_string.split(", ")
print(type(parts))
print(parts)
if ', ' in my_string:
    print("Aaron")